In [ ]:
# ВСПОМНИМ, С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ

# В этом модуле мы продолжим нашу работу с датасетом о продажах объектов недвижимости в Мельбурне и его пригородах. 
# Информацию об исходных данных вы можете найти здесь.

# В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

# удалили столбцы index и Coordinates;
# создали признак средней площади одной комнаты MeanRoomsSquare 
# и ввели коэффициент соотношения площади здания к площади участка — AreaRatio;
# преобразовали признак даты продажи Date в формат datetime и создали на его основе следующие столбцы: 
# номер месяц продажи (MonthSale), номер дня недели продажи (WeekdaySale), «признак-мигалку» выходного дня (Weekend);
# заменили признак года постройки здания YearBuilt на его возраст AgeBuilding;
# извлекли из признака адреса объекта Address новый признак подтипа улицы StreetType и удалили столбец с адресом;
# уменьшили число уникальных наименований агентств по недвижимости (SellerG), а также число пригородов (Suburb);
# выделили категориальные признаки и преобразовали их в тип данных сategory;
# заменили сокращённые названия категорий признака типа объекта Type на их полные названия (h — house, t —  townhouse, u — unit).

# Прочитаем датасет и выведем первые пять его строк:

import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

In [ ]:
# Файл подгрузился верно, однако есть один нюанс. Давайте посмотрим на информацию о столбцах с помощью метода info():

print(melb_df.info())

In [ ]:
# ЗАДАНИЕ 1.1
# Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. 
# Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
quarters = melb_df['Date'].dt.quarter
print(quarters.value_counts().iloc[1])

# ЗАДАНИЕ 1.2
# Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, 
# исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
# В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
max_unique_count = 150 
for col in melb_df.columns: 
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: 
        melb_df[col] = melb_df[col].astype('category')
print(melb_df.info())

### 2. Сортировка данных в DataFrame

In [ ]:
# Добавить страницу в мои закладки
# Часто бывает так, что таблицу или промежуточный результат операций с ней необходимо отсортировать по какому-то критерию. 
# Например, для отчётности вам необходимо предоставить список проданных объектов недвижимости, отсортированный по возрастанию цены 
# или дате продажи. Рассмотрим основные подходы к решению таких задач.

# МЕТОД SORT_VALUES()
# Для сортировки значений в DataFrame по значениям одного или нескольких столбцов используется метод sort_values().
# Основные параметры метода sort_values()

# СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА
# Приведём несколько примеров сортировки нашей таблицы с недвижимостью.
# Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):
melb_df.sort_values(by='Price').head(10)

In [ ]:
# Мы вывели десять строк таблицы, чтобы убедиться в верном порядке сортировки. 
# Также обратите внимание на индексы таблицы — их значения сохранились из исходной таблицы.

# А теперь отсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date). 
# Для этого выставим параметр ascending на False:
melb_df.sort_values(by='Date', ascending=False)

In [ ]:
# СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

# Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр by в виде списка. 
# При этом важно обращать внимание на порядок следования столбцов.

# Так, например, отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance), 
# а затем — по возрастанию цены объекта (Price). Для того чтобы вывод был более наглядным, выделим каждую десятую строку 
# из столбцов Distance и Price результирующей таблицы:
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

In [ ]:
# КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ

# А теперь рассмотрим применение сортировки на практике.

# Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов, 
# у которых площадь участка существенно больше площади здания, чтобы понять, 
# как часто компания справляется с таким сложным видом объектов. 
# Объекты, проданные в один и тот же день, мы бы хотели сортировать по значению коэффициента соотношения площадей. 

# Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, у которых коэффициент соотношения площадей 
# здания и участка (AreaRatio) меньше -0.8. Результат отсортируем по дате продажи (Date) в порядке возрастания, 
# а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, 
# установив параметр ignore_index на True. Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio:

mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

In [ ]:
# ЗАДАНИЕ 2.2
# Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. 
# Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.
int(melb_df.sort_values(
    by='AreaRatio', 
    ignore_index=True,
    ascending=False
).loc[1558, 'BuildingArea'])

# ЗАДАНИЕ 2.3
# Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. 
# Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). 
# Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
int(melb_df[mask1&mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price'])

### 3. Группировка данных в DataFrame

In [ ]:
# Одна из основных задач анализа данных — это группировка данных и сравнение показателей в группах. Например, 
# нам необходимо сравнить средний уровень заработной платы в зависимости от пола/уровня образования. 
# Или же мы хотим проследить, какая группа клиентов приносит нам наибольший доход, чтобы направить своё внимание на эту группу.

# В некоторых случаях группировки может быть достаточно, чтобы ответить на вопросы бизнеса. 
# В других случаях это может стать первым шагом в более сложном анализе. 
# Так, например, благодаря группировке мы можем выявлять признаки, которые не несут статистической значимости, или признаки, 
# которые вносят наибольший вклад. 

# Так или иначе, владение группировкой — важный навык, который открывает новые возможности по работе с данными.

# МЕТОД GROUPBY()

# В библиотеке Pandas для группировки данных по одному или нескольким признакам можно использовать метод groupby().

# Основные параметры метода groupby()
# Метод groupby() возвращает объект DataFrameGroupBy, который хранит в себе информацию о том, 
# какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 

# Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (mean, median, sum и т. д.), 
# чтобы рассчитывать показатели внутри каждой группы.

# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

# Рассмотрим группировку данных на примере нашей таблицы с недвижимостью.

# Применим агрегирующую функцию среднего к результату работы groupby(). 
# В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):
melb_df.groupby(by='Type').mean()

In [ ]:
# Как правило, нам не нужна информация обо всех столбцах, 
# поэтому агрегирующие методы можно применять только к интересующему нас столбцу. 
# Например, давайте сравним средние цены на объекты в зависимости от их типа:
melb_df.groupby('Type')['Price'].mean()

# Примечание. Обратите внимание, что, так как мы считаем только один показатель (среднее) для одного столбца, 
# в результате мы получаем объект Series.
# Из этой маленькой таблицы видно, что наибольшей средней ценой обладают объекты типа house (дома, коттеджи, виллы). 
# Следовательно, можно сделать вывод, что тип постройки является значимым фактором при определении цены объекта недвижимости.

In [ ]:
# Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.
# Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона. 
# Результат отсортируем по убыванию расстояния:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

In [ ]:
# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

# Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(), 
# который принимает список строк с названиями агрегаций.

# Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, 
# а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). 
# Результат отсортируем по количеству продаж в порядке убывания:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

# Примечание. Обратите внимание, что, так как мы считаем несколько показателей для одного столбца, 
# в результате мы получаем объект DataFrame.

# В результате применения метода agg(), в который мы передали список с названиями интересующих нас агрегирующих функций, 
# мы получаем DataFrame со столбцами count, mean и max, где для каждого месяца рассчитаны соответствующие параметры. 
# Результат сортируем по столбцу count.

In [ ]:
# Примечание. Если вам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, 
# вы можете воспользоваться методом agg(), передав в качестве его параметра строку 'describe':
melb_df.groupby('MonthSale')['Price'].agg('describe')

In [ ]:
# После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. 
# Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, 
# в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 

# Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set, 
# чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

In [ ]:
# ЗАДАНИЕ 3.1
# Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. 
# В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

# ЗАДАНИЕ 3.2
# Какой регион имеет наименьшую протяжённость по географической широте (Lattitude)?
# Для ответа на этот вопрос рассчитайте стандартное отклонение широты для каждого региона.
# В качестве ответа запишите название этого региона.
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

# ЗАДАНИЕ 3.3
# Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
# Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
# Не забудьте перевести даты в формат datetime.
date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= melb_df['Date']) & (melb_df['Date'] <= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

### 4. Сводные таблицы

In [ ]:
# Сводные таблицы — это распространённый инструмент для агрегации данных.

# Сводная таблица принимает на вход данные из отдельных столбцов и группирует их. 
# В результате получается новая таблица, которая позволяет увидеть многомерное обобщение данных. 
# Таким образом, благодаря сводным таблицам мы можем оценить зависимость между двумя и более признаками данных.

# Мы чаще сталкиваемся со сводными таблицами, чем с обычными, в плоском виде, так как сводные таблицы удобнее для анализа 
# и быстрых выводов, а также позволяют увидеть более общие зависимости между признаками, нежели простая группировка данных.

# Инструмент сводных таблиц также широко популярен среди тех, кто использует Excel или какие-либо BI-системы.

# МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

# На самом деле мы с вами уже строили простейшие одномерные сводные таблицы с помощью метода groupby — мы рассматривали 
# сводную таблицу в контексте группировки по одному признаку. 

# Например, мы уже умеем строить таблицу, которая показывает зависимость медианной цены и площади здания от числа комнат:
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

In [ ]:
# Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type). 
# Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.
melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

In [ ]:
# В результате выполнения такого кода мы получаем Series, которая обладает несколькими уровнями индексов: 
# первый уровень — число комнат, второй уровень — тип здания. Такая организация индексов называется иерархической. 
# Вычисление параметра (средней цены) происходит во всех возможных комбинациях признаков.

# Для того, чтобы финальный результат был представлен в виде сводной таблицы 
# (первый группировочный признак по строкам, а второй — по столбцам), а не в виде Series с иерархическими индексами, 
# к результату чаще всего применяют метод unstack(), который позволяет переопределить вложенный индекс в виде столбцов таблицы:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

In [ ]:
# МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

# На самом деле метод groupby редко используется при двух параметрах, 
# так как для построения сводных таблиц существует специальный и более простой метод — pivot_table().

# Основные параметры метода pivot_table()
# Давайте построим ту же самую таблицу, но уже с использованием метода pivot_table. 
# В качестве параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms, 
# а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее). 
# Дополнительно заменим пропуски в таблице на значение 0. Финальный результат 
# для наглядности вывода округлим с помощью метода round() до целых.
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

In [ ]:
# Несложно понять, что метод pivot_table() имеет преимущество перед группировкой по нескольким критериям. 
# Оно заключается в наличии специальных аргументов для строк и столбцов сводной таблицы, 
# благодаря чему уменьшается вероятность запутаться при построении более сложных (многомерных) сводных таблиц,
# о которых мы поговорим далее.

# А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. 
# Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), 
# а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, 
# если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count',
)

In [ ]:
# Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) 
# и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, 
# нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую сводную таблицу, 
# где пропущенные значения заменим на 0:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

In [ ]:
# МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ

# До этого мы рассматривали, как некоторый статистический показатель может зависеть от двух признаков. 
# Однако, как уже упоминалось, сводные таблицы позволяют наблюдать зависимость и от большего числа признаков. 
# Такие сводные таблицы называются многомерными. 

# Для того чтобы исследовать зависимость от большего числа признаков, 
# можно передать список признаков в параметр index или параметр columns.

# Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) 
# и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов 
# будет стоять медианная цена объекта (Price):
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

In [ ]:
# ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ

# Как получить доступ к данным или произвести фильтрацию в сложной сводной таблице, где есть дополнительные индексы?
# Давайте рассмотрим, что собой представляют столбцы сложной сводной таблицы.
# Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

# Выведем её столбцы с помощью атрибута columns:
pivot.columns

In [ ]:
# В результате мы получаем объект MultiIndex. Этот объект хранит в себе шесть комбинаций пар столбцов 
# (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

# Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. 
# Чтобы получить доступ к определённому столбцу, вы должны сначала обратиться к столбцу, который находится уровнем выше.

# Так, из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit, 
# просто последовательно обратившись по имени столбцов:
print(pivot['mean']['unit'])

In [ ]:
# Аналогично производится и фильтрация данных. Например, если нам нужны регионы, 
# в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
print(filtered_pivot)

In [ ]:
# Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index и обернуть результат в список:
print(list(filtered_pivot.index))

In [ ]:
# Примечание. На самом деле мультииндексные таблицы можно создавать и вручную. Давайте посмотрим на синтаксис данной конструкции:
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
print(mser)

# В данном примере мы создаём объект Series со вложенными индексами. 
# Мы передаём в качестве индексов Series вложенный список, где первый список задаёт внешний уровень вложенности, 
# а второй список — внутренний уровень вложенности. Значения Series — случайные числа от 0 до 1, 
# сгенерированные функцией np.random.rand() (ваши значения могут отличаться).
# Если посмотреть на индексы Series, можно увидеть, что они являются мультииндексами:
print(mser.index)

In [ ]:
# Аналогично создаются DataFrame со вложенными признаками 
# (вложенными столбцами) — для этого вложенный список передаётся в параметр columns при инициализации таблицы:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
print(mframe)

In [ ]:
# ЗАДАНИЕ 4.2
# Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания 
# от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая?
# В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.
pivot = melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)
print(pivot)

# ЗАДАНИЕ 4.3
# Составьте сводную таблицу, которая показывает зависимость средней цены объекта недвижимости (Price) 
# от риелторского агентства (SellerG) и типа здания (Type).
# Во вновь созданной таблице найдите агентство, у которого средняя цена для зданий типа unit максимальна. 
# В качестве ответа запишите название этого агентств
pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='mean',
)
max_unit_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_unit_price].index[0])

### 6. Объединение DataFrame: concat

In [ ]:
# Добавить страницу в мои закладки
# Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы ratings1 и ratings2 по строкам.

# Для этого воспользуемся встроенной функцией Pandas concat(), которая позволяет склеивать 
# (конкатенировать) таблицы как по строкам, так и по столбцам.

# Кликните на плашку, чтобы увидеть информацию ↓

# Основные параметры функции concat()
# Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.

# Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. 
# Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию равен 0, 
# объединение происходит по строкам, поэтому не трогаем его. 

# Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. 
# Поэтому её вызов осуществляется как pd.concat(...).

ratings1 = pd.read_csv('data/ratings1.csv', sep=',')
ratings2 = pd.read_csv('data/ratings2.csv', sep=',')
dates = pd.read_csv('data/movies.csv', sep=',')
movies = pd.read_csv('data/movies.csv', sep=',')

ratings = pd.concat([ratings1, ratings2])
print(ratings)

In [ ]:
# В результате мы увеличили первую таблицу, добавив снизу строки второй таблицы.

# На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим на индексы последних строк таблицы, 
# то увидим, что их нумерация не совпадает с количеством строк. Это может привести к некорректному 
# объединению таблиц по ключевым столбцам на следующем этапе решения нашей задачи.

# Это связано с тем, что по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, 
# а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
print(ratings)

In [ ]:
# Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах ratings и dates, 
# ведь нам предстоит вертикально склеить их между собой:

print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

In [ ]:
# На самом деле очень просто: при выгрузке данных информация об оценках какого-то
# пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. 
# В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2:

print(ratings1.tail(1))
print(ratings2.head(1))

In [ ]:
# Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), 
# который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, 
# выставив параметр ignore_index в методе drop_duplicates() на значение True:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

# Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. 
# Для этого конкатенируем таблицы ratings и dates по столбцам:
ratings_dates = pd.concat([ratings, dates], axis=1)
print(ratings_dates.tail(7))

### 7. Объединение DataFrame: join, merge

У таблиц ratings и movies есть общий столбец movieId, который каждому фильму из таблицы movies ставит 
в соответствие поставленные ему оценки из таблицы ratings. Мы хотим объединить их в единую структуру согласно этому соответствию. 
Объединения такого рода часто называют объединением по ключевому столбцу.

Однако прежде чем мы перейдём к дальнейшей работе с нашими таблицами о фильмах, мы должны рассмотреть основные типы объединения таблиц.

### ТИПЫ ОБЪЕДИНЕНИЙ

### inner (внутреннее)

При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах.

Аналогия в теории множеств
Строки, для которых совпадение не было найдено, удаляются.

### outer (внешнее)

Данный тип делится на три подтипа:

full — используется как outer по умолчанию, объединяет все варианты в обеих таблицах.

Аналогия в теории множеств
left — для всех записей из «левой» таблицы (например, ratings) ведётся поиск соответствий в «правой» (например, movies). 
В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.

Аналогия в теории множеств
right — аналогично предыдущему, но остаются значения только из «правой» таблицы.

In [ ]:
# МЕТОД ОБЪЕДИНЕНИЯ JOIN

# Для объединения двух таблиц по индексам используется метод DataFrame join(). 
# Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это movieId).

# Если использовать метод join() «в лоб» (без указания ключевого столбца), то объединение произойдёт, 
# как и задумано — по индексам двух таблиц согласно установленному типу объединения.

# Проверим это, объединив таблицы типом left. Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, 
# чтобы избежать ошибки:

joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
print(joined_false)

In [ ]:
# При объединении таблиц по индексам в результирующую таблицу попали все строки из «левой» таблицы, 
# а недостающие строки из «правой» были заполнены пропусками. Так работает тип объединения left.

# Попробуйте изменить тип объединения, чтобы посмотреть на разницу результирующих таблиц.

# Обратите внимание, что в данном случае у нас получилось два столбца, соответствующих идентификатору фильма: 
# один — из «левой» таблицы (movieId), а другой — из «правой» (movieId_right).

# Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. 
# Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), 
# необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. 
# Это можно сделать с помощью метода set_index(). Также необходимо указать название ключа в параметре on.

joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
print(joined.head())

In [ ]:
# МЕТОД ОБЪЕДИНЕНИЯ MERGE

# Аналогично предыдущему, метод merge() предназначен для слияния двух таблиц по ключевым столбцам или по индексам. 
# Однако, в отличие от join(), метод merge() предлагает более гибкий способ управления объединением, 
# благодаря чему является более популярным.

# Кликните на плашку, чтобы увидеть информацию ↓

# Основные параметры метода merge()
# Метод merge() в первую очередь предназначен для слияния таблиц по заданным ключам, поэтому он не требует 
# установки ключевых столбцов в качестве индекса присоединяемой таблицы. Кроме того, данный метод позволяет 
# объединять даже таблицы с разноимёнными ключами. Таким образом, merge() проще в использовании и более многофункционален, 
# чем схожие методы.

# Посмотрим на метод merge() в действии. Произведём слияние наших таблиц и получим ту же таблицу, что и ранее:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
print(merged.head())

In [ ]:
# Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей таблице merged:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

In [ ]:
# ОСОБЕННОСТИ ИСПОЛЬЗОВАНИЯ MERGE()

# Возникает вопрос: почему мы выбрали тип объединения left, а не full, например?

# Найти ответ нам поможет пример. Объединим ratings_dates с movies по ключевому столбцу movieId, 
# но с параметром how='outer' (full outer) и выведем размер таблицы, а также её «хвост»:
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print('Число строк в таблице merged2: ', merged2.shape[0])
print(merged2.tail())

In [ ]:
# Оказывается, в таблице movies содержались фильмы, которым ещё не были выставлены оценки. 
# В результате объединения типом full outer информация о фильмах перенеслась из таблицы movies в результирующую таблицу. 
# Однако, поскольку оценки фильмам ещё не были выставлены, соответствующие столбцы таблицы ratings_dates заполнились пропусками (NaN). 
# Такие фильмы были записаны в конец таблицы.

# Важно! Учитывайте такие нюансы при работе с несколькими таблицами и всегда проверяйте результат объединения.

# Метод merge() с внешним (outer) типом объединения может использоваться как аналог метода concat() 
# при объединении таблиц с одинаковой структурой (одинаковые количество и названия столбцов) по строкам. 
# В таком случае все одноимённые столбцы таблиц будут считаться ключевыми.

# Рассмотрим пример: объединим таблицы ratings1 и ratings2, как мы уже делали раньше, но теперь используем метод merge():
merge_ratings = ratings1.merge(ratings2, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
print(merge_ratings)

In [ ]:
# ЗАДАНИЕ 7.5
# 1. Сформируйте DataFrame merged, в котором в результате объединения purchase_df и items_df останутся модели, 
# которые учтены на складе и имели продажи.
# 2. Из таблицы merged найдите суммарную выручку, которую можно было бы получить от продажи всех товаров, 
# которые учтены на складе и имели продажи. Результат занесите в переменную income.
items_df = pd.DataFrame({
'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merged = items_df.merge(purchase_df, how='inner', on='item_id')
income = (merged['price'] * merged['stock_count']).sum()

print(f'merged: {merged}')
print()
print(f'income: {income}')

In [ ]:
orders_df = pd.read_csv('data/orders.csv', sep=',')
product_df = pd.read_csv('data/products.csv', sep=',')
ratings_movies_df = pd.read_csv('data/ratings_movies.csv', sep=',')


#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None



# ЗАДАНИЕ 8.1
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?
joined['year_release'] = joined['title'].apply(get_year_release)
joined.info()

# ЗАДАНИЕ 8.2
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.
mask = joined['year_release'] == 1999
joined[mask].groupby('title')['rating'].mean().sort_values()

# ЗАДАНИЕ 8.3
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
mask = joined['year_release'] == 2010
joined[mask].groupby('genres')['rating'].mean().sort_values()

# ЗАДАНИЕ 8.4
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) жанров (genres) фильмов? 
joined.groupby('userId')['genres'].nunique().sort_values(ascending=False)

# ЗАДАНИЕ 8.5
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
joined.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])

# ЗАДАНИЕ 8.6
# Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
mask = joined['year_release'] == 2018
grouped = joined[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)

# ЗАДАНИЕ 8.7
# Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, 
# которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. 
# Выберите верные варианты ответа, исходя из построенной таблицы:
joined['date'] = pd.to_datetime(joined['date'])
joined['year_rating'] = joined['date'].dt.year
pivot = joined.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
print(pivot)

# ЗАДАНИЕ 8.8
# Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?
orders_products = orders_df.merge(
    product_df, 
    left_on='ID товара',
    right_on='Product_ID',
    how='left')
orders_products.tail(1)['Order ID']

# ЗАДАНИЕ 8.9
# На какой товар была произведена отмена?
orders_products[orders_products['Отменен'] == 'Да']['Name']

# ЗАДАНИЕ 8.10
# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
# В ответ запишите идентификатор этого покупателя (ID Покупателя).
# Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество'] 
orders_products[orders_products['Оплачен'] == 'Да'].groupby('ID Покупателя')['Profit'].sum().sort_values(ascending=False)